In [2]:
import numpy as np
import pandas as pd
import os
import shutil
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from collections.abc import Callable
import ipympl
import math
import random
import time
import tracemalloc
from functools import wraps
import sys
import gc
%matplotlib ipympl

# Implementacja Metody Gaussa

In [3]:
def gauss(A, b, precision):
  A = A.astype(precision)
  b = b.astype(precision)
  n = A.shape[0]
  for i in range(n):
    
    _max_i = np.argmax(np.abs(A[i:,i])) + i
    A[[i, _max_i], :] = A[[_max_i, i], :]
    b[i], b[_max_i] = b[_max_i], b[i]
    
    b[i] /= A[i,i]
    A[i,:] /= A[i,i]
    
    for j in range(n):
      if(j != i):
        b[j] -= b[i] * A[j,i]
        A[j,:] -= A[i,:] * A[j,i]
        
  return b


In [4]:
A = np.array([[2,1,2],
              [1,2,1],
              [3,1,-1]])
b = np.array([10,8,2])
gauss(A,b, np.float64)

array([1., 2., 3.])

# Implementacja metody Thomasa

In [5]:
# http://www.industrial-maths.com/ms6021_thomas.pdf <3

def thomas(A, b, precision):
  A = A.astype(precision)
  b = b.astype(precision)
  n = A.shape[0]
  
  b[0] /= A[0,0]
  A[0,:] /= A[0,0]
  for i in range(1,n):
    b[i] -= b[i-1] * A[i,i-1]
    A[i,:] -= A[i-1,:] * A[i,i-1]
    b[i] /= A[i,i]
    A[i,:] /= A[i,i]
    
  for i in range(n-2,-1,-1):
    b[i] -= b[i+1] * A[i,i+1]
    
  return b

In [6]:

# For n x 3 matrix
def thomas_memory(A, b, precision):
  
  A = A.astype(precision)
  b = b.astype(precision)
  n = A.shape[0]
  
  b[0] /= A[0,1]
  A[0,:] /= A[0,1]
  for i in range(1,n):
    b[i] -= b[i-1] * A[i,0]
    # A[i,1:] -= A[i-1,1:] * A[i,0]
    A[i,1] -= A[i-1,2] * A[i,0]
    A[i,0] -= (A[i-1,1] * A[i,0])
    b[i] /= A[i,1]
    A[i,:] /= A[i,1]
    
  for i in range(n-2,-1,-1):
    b[i] -= b[i+1] * A[i,2]
  
  return b

In [7]:
A = np.array([[2,-1,0,0],
              [-1,2,-1,0],
              [0,-1,2,-1],
              [0,0,-1,1]])
A_min = np.array([
  [0,2,-1],
  [-1,2,-1],
  [-1,2,-1],
  [-1,1,0],
])
b = np.array([0,0,1,0])
print(thomas(A,b,np.float32))
thomas_memory(A_min,b,np.float32)

[1.0000004 2.0000007 3.000001  3.000001 ]


array([1.0000004, 2.0000007, 3.000001 , 3.000001 ], dtype=float32)

## Wyznaczanie Błędu

In [8]:
def maximum_error(A,B):
  n = len(A)
  return max([np.abs(A[i] - B[i]) for i in range(n)])

## Pomiar Czasu

In [9]:
def get_time(func, *args, **kwargs):
  gc.collect()
  start_time = time.perf_counter()
  func(*args, **kwargs)
  end_time = time.perf_counter()
  total_time = end_time - start_time
  return total_time

## Wyznaczanie Macierzy

In [10]:

def get_X_vector(n, precision):
  random.seed(42)
  X = np.array([1 if random.randint(1,2) == 1 else -1 for _ in range(n)], dtype=precision)
  return X

def task_1_matrix(n, X, precision):
  A = np.array([[1/(i+j+2-1) if i != 0 else 1 for j in range(n)] for i in range(n)], dtype=precision)
  b = A @ X
  return A, b 


def task_2_matrix(n, X, precision):
  A = np.array([[(2*(i+1))/(j+1) if j >= i else (2*(j+1))/(i+1) for j in range(n)] for i in range(n)], dtype=precision)
  b = A @ X
  return A, b 
  
  
k,m = 7, 3

def element_tridiagonal(i,j, k, m):
  if(i == j):
    return k
  if(j == i+1):
    return 1/(i+1+m)
  if(j == i-1):
    return k/(i+1+m+1)
  return 0

def element_tridiagonal_memory(i,j, k, m, n):
  if(i == 0 and j ==0):
    return 0
  if(i == n-1 and j==2):
    return 0
  if(j == 1):
    return k
  if(j == 2):
    return 1/(i+1+m)
  if(j == 0):
    return k/(i+1+m+1)
  return 0

def task_3_matrix_memory(n,X,precision):
  A = np.array([[element_tridiagonal(i,j,k,m) for j in range(n)] for i in range(n)], dtype=precision)
  A_alt = np.array([[element_tridiagonal_memory(i,j,k,m, n) for j in range(3)] for i in range(n)], dtype=precision)
  b = A @ X
  return A_alt, b

def task_3_matrix(n,X,precision):
  A = np.array([[element_tridiagonal(i,j,k,m) for j in range(n)] for i in range(n)], dtype=precision)
  b = A @ X
  return A, b
  
  
N = 3
PRECISION = np.float64

X = get_X_vector(N, PRECISION)
# A, b = task_2_matrix(N, X, PRECISION)

A, b = (task_3_matrix_memory(N, X, PRECISION))
print(A)
result = thomas_memory(A, b, PRECISION)
np.set_printoptions(precision=1000, suppress=True)
print(result)


[[0.         7.         0.25      ]
 [1.16666667 7.         0.2       ]
 [1.         7.         0.        ]]
[ 1.  1. -1.]


# Wizualizacja

In [11]:
def plot_function(
  x_values: np.ndarray[np.float64],
  func_to_plot: Callable[[np.float64], np.float64],
  nodes: np.ndarray | None = None,
  x_lim: tuple[float, float] | None = None,
  y_lim: tuple[float, float] | None = None,
  x_scale: str = 'linear',
  y_scale: str = 'linear',
  function_name: str = 'Funkcja bazowa',
  nodes_name: str = '',
  title: str = 'Wizualizacja funkcji',
  linestyle: str = '-', 
  color: str | None = None,
  alpha: float = 1.0,
  draw_axes = False,
  yticks_num = None,
  xticks_num = None
):
  y_values = np.array([func_to_plot(x) for x in x_values])

  if(color):
    plt.plot(x_values, y_values, label=function_name,
            linestyle=linestyle, color=color, alpha=alpha)
  else:
    plt.plot(x_values, y_values, label=function_name,
            linestyle=linestyle, alpha=alpha)
  
  if nodes is not None:
    if(color):
      plt.scatter(nodes[:, 0], nodes[:, 1], color=color,
                  label=nodes_name, s=15, zorder=5)
    else:
      plt.scatter(nodes[:, 0], nodes[:, 1],
                  label=nodes_name, s=15, zorder=5)

  plt.xlabel('x')
  plt.ylabel('y')

  if x_lim:
    plt.xlim(x_lim)
  if y_lim:
    plt.ylim(y_lim)
  if(draw_axes):
    plt.axhline(0,color='black',lw=0.8)
    plt.axvline(0,color='black', lw=0.8)
  
  plt.xscale(x_scale)
  plt.yscale(y_scale)
  
  if(yticks_num is not None):
    yticks = np.linspace(*plt.ylim(), num=yticks_num)
    plt.yticks(yticks)
  if(xticks_num is not None):
    xticks = np.linspace(*plt.ylim(), num=xticks_num)
    plt.xticks(xticks)
  plt.legend()
  plt.title(title)
  plt.grid(True)


# Zadanie 1

In [12]:
FIRST_N = [i for i in range(2,21)]
SECOND_N = [i for i in range(2,200)]
PRECISIONS = [np.float32, np.float64]

os.makedirs('data', exist_ok=True)
os.makedirs('plots', exist_ok=True)
pd.set_option('display.float_format', '{:.5f}'.format)

def benchmark(matrix_func, n_range, precisions, method, task_name):
    for precision in precisions:
        precision_name = 'float32' if precision == np.float32 else 'float64'
        
        # Przygotowanie DataFrame do przechowywania wyników
        results = pd.DataFrame(columns=['n', 'error', 'condition_number', 'time'])
        
        for n in n_range:
            # Generowanie wektora X i macierzy układu
            X = get_X_vector(n, precision)
            A, b = matrix_func(n, X, precision)
            
            # Pomiar czasu rozwiązania
            start_time = time.perf_counter()
            X_approx = method(A.copy(), b.copy(), precision)
            end_time = time.perf_counter()
            solve_time = end_time - start_time
            
            # Wyznaczenie błędu
            error = maximum_error(X, X_approx)
            
            # Wyznaczenie uwarunkowania macierzy
            cond_number = np.linalg.cond(A, p=1)
            
            # Dodanie wyników do DataFrame
            results = pd.concat([results, pd.DataFrame({
                'n': [n],
                'error': [error],
                'condition_number': [cond_number],
                'time': [solve_time]
            })], ignore_index=True)
            
            # Wyświetlenie postępu
            print(f"Task: {task_name}, Precision: {precision_name}, n: {n}, Error: {error:.6e}, Cond: {cond_number:.6e}, Time: {solve_time:.6f}s")
        
        # Zapisanie wyników do pliku CSV
        filename = f"data/{task_name}_{precision_name}.csv"
        results.to_csv(filename, index=False, float_format='{:.4e}'.format)
        print(f"Results saved to {filename}")

def plot_results():
    

    """
    Funkcja tworząca wykresy na podstawie zapisanych danych.
    """
    # Lista plików z danymi
    files = [f for f in os.listdir('data') if f.endswith('.csv')]
    
    # Przygotowanie danych do wykresów porównawczych
    task1_float32 = pd.read_csv('data/task1_float32.csv')
    task1_float64 = pd.read_csv('data/task1_float64.csv')
    task2_float32 = pd.read_csv('data/task2_float32.csv')
    task2_float64 = pd.read_csv('data/task2_float64.csv')
    
    # Ustawienie stylu wykresów
    sns.set_style("whitegrid")
    plt.figure(figsize=(10, 6))
    
    # 1. Wykres błędu dla zadania 1
    print(max(task1_float64['error']))
    plt.figure(figsize=(10, 6))
    plt.semilogy(task1_float32['n'], task1_float32['error'], 'o-', label='float32', color="red")
    plt.semilogy(task1_float64['n'], task1_float64['error'], 'o-', label='float64', color='blue')
    plt.xlabel('Rozmiar macierzy (n)')
    plt.ylabel('Błąd maksymalny')
    plt.title('Błąd rozwiązania dla macierzy z zadania 1')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.gca().xaxis.set_major_locator(plt.MaxNLocator(integer=True))
    plt.savefig('plots/task1_error.png')
    
    # 2. Wykres czasu dla zadania 1
    plt.figure(figsize=(10, 6))
    plt.plot(task1_float32['n'], task1_float32['time'], 'o-', label='float32', color="red")
    plt.plot(task1_float64['n'], task1_float64['time'], 'o-', label='float64', color="blue")
    plt.xlabel('Rozmiar macierzy (n)')
    plt.ylabel('Czas rozwiązania [s]')
    plt.title('Czas rozwiązania dla macierzy z zadania 1')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.gca().xaxis.set_major_locator(plt.MaxNLocator(integer=True))
    plt.savefig('plots/task1_time.png')
    
    # 3. Wykres uwarunkowania dla zadania 1
    plt.figure(figsize=(10, 6))
    plt.semilogy(task1_float32['n'], task1_float32['condition_number'], 'o-', label='float32', color="red")
    plt.semilogy(task1_float64['n'], task1_float64['condition_number'], 'o-', label='float64', color="blue")
    plt.xlabel('Rozmiar macierzy (n)')
    plt.ylabel('Wskaźnik uwarunkowania')
    plt.title('Uwarunkowanie macierzy z zadania 1')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.gca().xaxis.set_major_locator(plt.MaxNLocator(integer=True))
    plt.savefig('plots/task1_condition.png')
    
    # 4. Wykres błędu dla zadania 2
    plt.figure(figsize=(10, 6))
    plt.semilogy(task2_float32['n'], task2_float32['error'], label='float32', color="red")
    plt.semilogy(task2_float64['n'], task2_float64['error'], label='float64', color="blue")
    plt.xlabel('Rozmiar macierzy (n)')
    plt.ylabel('Błąd maksymalny')
    plt.title('Błąd rozwiązania dla macierzy z zadania 2')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.gca().xaxis.set_major_locator(plt.MaxNLocator(integer=True))
    plt.savefig('plots/task2_error.png')
    
    # 5. Wykres czasu dla zadania 2
    plt.figure(figsize=(10, 6))
    plt.plot(task2_float32['n'], task2_float32['time'], label='float32', color="red")
    plt.plot(task2_float64['n'], task2_float64['time'], label='float64', color="blue")
    plt.xlabel('Rozmiar macierzy (n)')
    plt.ylabel('Czas rozwiązania [s]')
    plt.title('Czas rozwiązania dla macierzy z zadania 2')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.gca().xaxis.set_major_locator(plt.MaxNLocator(integer=True))
    plt.savefig('plots/task2_time.png')
    
    # 6. Wykres uwarunkowania dla zadania 2
    plt.figure(figsize=(10, 6))
    plt.semilogy(task2_float32['n'], task2_float32['condition_number'], label='float32', color="red")
    plt.semilogy(task2_float64['n'], task2_float64['condition_number'], label='float64', color="blue")
    plt.xlabel('Rozmiar macierzy (n)')
    plt.ylabel('Wskaźnik uwarunkowania')
    plt.title('Uwarunkowanie macierzy z zadania 2')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.gca().xaxis.set_major_locator(plt.MaxNLocator(integer=True))
    plt.savefig('plots/task2_condition.png')
    
    # 7. Porównanie uwarunkowania obu macierzy
    plt.figure(figsize=(10, 6))
    # Wybieramy tylko float64 dla przejrzystości
    plt.semilogy(task1_float64['n'], task1_float64['condition_number'], 'o-', label='Macierz 1', color='green')
    # Dla task2 ograniczamy do zakresu n task1 dla lepszego porównania
    task2_subset = task2_float64[task2_float64['n'] <= max(task1_float64['n'])]
    plt.semilogy(task2_subset['n'], task2_subset['condition_number'], 'o-', label='Macierz 2', color='orange')
    plt.xlabel('Rozmiar macierzy (n)')
    plt.ylabel('Wskaźnik uwarunkowania')
    plt.title('Porównanie uwarunkowania obu macierzy (float64)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.gca().xaxis.set_major_locator(plt.MaxNLocator(integer=True))
    plt.savefig('plots/comparison_condition.png')
    
    # 8. Wykres pełnego uwarunkowania macierzy 2 (dla dużego zakresu n)
    plt.figure(figsize=(10, 6))
    plt.semilogy(task2_float64['n'], task2_float64['condition_number'], 'o-', label='float64')
    plt.xlabel('Rozmiar macierzy (n)')
    plt.ylabel('Wskaźnik uwarunkowania')
    plt.title('Uwarunkowanie macierzy z zadania 2 (pełny zakres)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.gca().xaxis.set_major_locator(plt.MaxNLocator(integer=True))
    plt.savefig('plots/task2_condition_full.png')
    
    print("Wszystkie wykresy zostały zapisane w katalogu 'plots'")

# Kod do uruchomienia benchmarku
def run_benchmarks():
    # Zadanie 1 - metoda Gaussa dla mniejszego zakresu n (2-20)
    benchmark(task_1_matrix, FIRST_N, PRECISIONS, gauss, "task1")
    
    # Zadanie 2 - metoda Gaussa dla większego zakresu n (2-200)
    benchmark(task_2_matrix, SECOND_N, PRECISIONS, gauss, "task2")
    
    # Generowanie wykresów
    plot_results()

# Przykład użycia:
# run_benchmarks()


# ZADANIE 2

In [24]:


os.makedirs('data', exist_ok=True)
os.makedirs('plots', exist_ok=True)
pd.set_option('display.float_format', '{:.5f}'.format)

def benchmark(matrix_func, n_range, precisions, method, task_name):
    for precision in precisions:
        precision_name = 'float32' if precision == np.float32 else 'float64'
        
        # Przygotowanie DataFrame do przechowywania wyników
        results = pd.DataFrame(columns=['n', 'error', 'time'])
        
        for n in n_range:
            # Generowanie wektora X i macierzy układu
            tracemalloc.start()
            tracemalloc.reset_peak()
            snapshot1 = tracemalloc.take_snapshot()

            X = get_X_vector(n, precision)
            A, b = matrix_func(n, X, precision)
            
            size_of_array = A.nbytes / 1024
            # Pomiar czasu rozwiązania
            start_time = time.perf_counter()
            X_approx = method(A.copy(), b.copy(), precision)
            snapshot2 = tracemalloc.take_snapshot()
            stats = snapshot2.compare_to(snapshot1, 'filename')
            peak = sum(stat.size_diff for stat in stats)

            peak = peak / (1024)
            end_time = time.perf_counter()
            tracemalloc.stop()
            solve_time = end_time - start_time
            
            # Wyznaczenie błędu
            error = maximum_error(X, X_approx)
            
            # Wyznaczenie uwarunkowania macierzy
            # cond_number = np.linalg.cond(A, p=1)
            
            # Dodanie wyników do DataFrame
            results = pd.concat([results, pd.DataFrame({
                'n': [n],
                'error': [error],
                'time': [solve_time],
                'mem': [peak],
                'arr_size': [size_of_array]
            })], ignore_index=True)
            
            # Wyświetlenie postępu
            print(f"Task: {task_name}, Precision: {precision_name}, n: {n}, Error: {error:.6e}, Time: {solve_time:.6f}s, Memory: {peak:.6f}")
        
        # Zapisanie wyników do pliku CSV
        filename = f"data/{task_name}_{precision_name}.csv"
        results.to_csv(filename, index=False, float_format='{:.4e}'.format)
        print(f"Results saved to {filename}")

def plot_results():
  """
  Tworzy wykresy błędów, czasu i pamięci dla każdego benchmarku,
  oraz wykresy porównawcze między benchmarkami i typami precyzji.
  """

  import os
  import seaborn as sns
  import matplotlib.pyplot as plt

  sns.set_style("whitegrid")

  tasks = ['thomas_mem', 'thomas', 'gauss']
  task_names = ['Metoda Thomasa - n × 3', 'Metoda Thomasa - n × n', 'Metoda Gaussa - n × n']
  precisions = ['float32', 'float64']
  metrics = ['error', 'time', 'mem']
  metric_labels = {
    'error': 'Błąd maksymalny',
    'time': 'Czas rozwiązania [s]',
    'mem': 'Użycie pamięci [KB]'
  }

  data = {}

  # Wczytaj dane
  for task in tasks:
    data[task] = {}
    for precision in precisions:
      filename = f"data/{task}_{precision}.csv"
      if not os.path.exists(filename):
        print(f"Plik {filename} nie istnieje. Pomijanie.")
        continue
      data[task][precision] = pd.read_csv(filename)

  # Wykresy pojedyncze dla każdego benchmarku
  for i,task in enumerate(tasks):
    for metric in metrics:
      plt.figure(figsize=(10, 6))
      for precision in precisions:
        if precision in data[task]:
          plt.scatter(
            data[task][precision]['n'],
            data[task][precision][metric],
            label=precision,
            s=2,
            marker='o' if precision == 'float32' else 's'
          )
      plt.xlabel('Rozmiar macierzy (n)')
      plt.ylabel(metric_labels[metric])
      plt.title(f'{metric_labels[metric]} dla {task_names[i]}')
      plt.legend()
      plt.grid(True)
      plt.tight_layout()
      plt.savefig(f'plots/{task}_{metric}.png')
      plt.close()

  # Porównanie między benchmarkami dla float32 i float64 osobno
  for precision in precisions:
    for metric in metrics:
      plt.figure(figsize=(10, 6))
      for i,task in enumerate(tasks):
        if precision in data[task]:
          plt.scatter(
            data[task][precision]['n'],
            data[task][precision][metric],
            label=task_names[i],
            s=2
          )
      plt.xlabel('Rozmiar macierzy (n)')
      plt.ylabel(metric_labels[metric])
      plt.title(f'{metric_labels[metric]} - porównanie metod ({precision})')
      plt.legend()
      plt.grid(True)
      plt.tight_layout()
      plt.savefig(f'plots/comparison_{precision}_{metric}.png')
      plt.close()

  # Porównanie float32 vs float64 na jednym wykresie dla każdego benchmarka
  for i,task in enumerate(tasks):
    for metric in metrics:
      plt.figure(figsize=(10, 6))
      for precision in precisions:
        if precision in data[task]:
          plt.scatter(
            data[task][precision]['n'],
            data[task][precision][metric],
            label=f'{task_names[i]} - {precision}',
            s=2
          )
      plt.xlabel('Rozmiar macierzy (n)')
      plt.ylabel(metric_labels[metric])
      plt.title(f'{metric_labels[metric]} - {task_names[i]} (float32 i float64)')
      plt.legend()
      plt.grid(True)
      plt.tight_layout()
      plt.savefig(f'plots/{task}_{metric}_both_precisions.png')
      plt.close()

  print("Wszystkie wykresy zostały zapisane w katalogu 'plots'.")


# Kod do uruchomienia benchmarku

N = [i for i in range(2,501)]
# SECOND_N = [i for i in range(2,200)]
PRECISIONS = [np.float32, np.float64]
def run_benchmarks():
    # Zadanie 1 - metoda Gaussa dla mniejszego zakresu n (2-20)
    # benchmark(task_3_matrix_memory, N, PRECISIONS, thomas_memory, "thomas_mem")
    
    # Zadanie 2 - metoda Gaussa dla większego zakresu n (2-200)
    # benchmark(task_3_matrix, N, PRECISIONS, thomas, "thomas")
    
    # Zadanie 2 - metoda Gaussa dla większego zakresu n (2-200)
    # benchmark(task_3_matrix, N, PRECISIONS, gauss, "gauss")
    
    # Generowanie wykresów
    plot_results()

# Przykład użycia:
run_benchmarks()


Wszystkie wykresy zostały zapisane w katalogu 'plots'.
